In [ ]:
from transformers import AutoTokenizer, AutoModel
from qalsadi.lemmatizer import Lemmatizer
tokenizer = AutoTokenizer.from_pretrained("arcee-ai/Meraj-Mini")
model = AutoModel.from_pretrained("arcee-ai/Meraj-Mini")
lemmatizer = Lemmatizer()
device = "cpu"
model.to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
import requests
import torch
request = requests.get('https://naama.sa/services/getservicesearch')
data = request.json()

print(data)
print(len(data))

In [ ]:
def lemmatize_text(text):
    return " ".join(lemmatizer.lemmatize(word) for word in text.split())


In [ ]:

def search_query(query, data, keys=['value','desc']):
    query_lemma = lemmatizer.lemmatize(query)
    results = []
    seen_ids = set()
    for item in data:
        for key in keys:
            if key in item:
                item_lemmas = lemmatize_text(item[key]).split()
                if query_lemma in item_lemmas and item['id'] not in seen_ids:
                    results.append(item)
                    seen_ids.add(item['id'])
                    break
    return results



In [ ]:
def embed_batch(texts, batch_size=512, device="cuda"):
    embeddings_list = []
    model.to(device)
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu()
            embeddings_list.append(batch_embeddings)
    embeddings = torch.cat(embeddings_list, dim=0)
    return embeddings


In [ ]:

query = "قطوه"

results = search_query(query, data)

texts_to_embed = [item['value'] for item in results]


device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = embed_batch(texts_to_embed, device=device)

print(" results:", len(results))
for r in results:
    print("-", r['value'])
